In [1]:
# !pip install --upgrade --quiet  langchain langchain-community llama-cpp-python langchain_huggingface wikipedia faiss-cpu faiss-gpu
# https://github.com/benbot/react.gbnf/tree/main

^C


In [1]:
import os
from llama_cpp import Llama

model_name_or_path ="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF"
model_file ="Meta-Llama-3.1-8B-Instruct-Q6_K.gguf"
n_ctx = None
model_name_or_path =model_name_or_path
model_file =model_file
user_n_ctx=512 if n_ctx is None else n_ctx
print(f"first n_ctx is set to {user_n_ctx}")

llm = Llama.from_pretrained(
                        repo_id=model_name_or_path,
                        filename=model_file,#*q8_0.gguf",
                        verbose=True,
                        n_ctx=user_n_ctx
                    )

first n_ctx is set to 512


llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from C:\Users\Admin\.cache\huggingface\hub\models--lmstudio-community--Meta-Llama-3.1-8B-Instruct-GGUF\snapshots\8601e6db71269a2b12255ebdf09ab75becf22cc8\.\Meta-Llama-3.1-8B-Instruct-Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str      

In [2]:
llm.model_path

'C:\\Users\\Admin\\.cache\\huggingface\\hub\\models--lmstudio-community--Meta-Llama-3.1-8B-Instruct-GGUF\\snapshots\\8601e6db71269a2b12255ebdf09ab75becf22cc8\\.\\Meta-Llama-3.1-8B-Instruct-Q6_K.gguf'

In [3]:
llm.n_ctx

<bound method Llama.n_ctx of <llama_cpp.llama.Llama object at 0x00000212F32D9280>>

In [4]:
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
examples =[{"input": "what is capital of India ?", 
        "answer": "The capital of India is Delhi"},
        {"input": "what is capital of France ?", 
        "answer": "The capital of France is Paris"},
        {"input": "what is capital of Russia ?", 
        "answer": "The capital of Russia is Moscow"},
        {"input": "what is capital of Canada ?", 
        "answer": "The capital of Canada is Ottawa"},
        {"input": "what is capital of Nepal ?", 
        "answer": "The capital of India is Kathmandu"},
        {"input": "How many grand slam titles did roger federer win?", 
        "answer": "20"},
        {"input": "Who is the first captain of indian cricket team?", 
        "answer": "c k nayudu"},
        {"input": "Who is the first first man to land on moon?", 
        "answer": "neil armstrong"},
        {"input": "what color is a mango?", 
        "answer": "yellow"},
        ]
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:

from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    embeddings,
    FAISS,
    k=2,
    input_keys=["input"],
    )


example_selector.vectorstore.search("what year did indian cricket team won first world cup?", search_type = "mmr", k=2)

[Document(metadata={'input': 'Who is the first captain of indian cricket team?', 'answer': 'c k nayudu'}, page_content='Who is the first captain of indian cricket team?'),
 Document(metadata={'input': 'How many grand slam titles did roger federer win?', 'answer': '20'}, page_content='How many grand slam titles did roger federer win?')]

In [9]:
example_selector.vectorstore.search("what colour is the strawberry?", search_type = "mmr", k=2)

[Document(metadata={'input': 'what color is a mango?', 'answer': 'yellow'}, page_content='what color is a mango?'),
 Document(metadata={'input': 'what is capital of France ?', 'answer': 'The capital of France is Paris'}, page_content='what is capital of France ?')]

In [13]:
example_selector.vectorstore.search("what is capital of germany?", search_type = "mmr", k=4)

[Document(metadata={'input': 'what is capital of France ?', 'answer': 'The capital of France is Paris'}, page_content='what is capital of France ?'),
 Document(metadata={'input': 'what is capital of Nepal ?', 'answer': 'The capital of India is Kathmandu'}, page_content='what is capital of Nepal ?'),
 Document(metadata={'input': 'what is capital of Russia ?', 'answer': 'The capital of Russia is Moscow'}, page_content='what is capital of Russia ?'),
 Document(metadata={'input': 'what is capital of Canada ?', 'answer': 'The capital of Canada is Ottawa'}, page_content='what is capital of Canada ?')]

In [16]:
import os
# from grammar import grammar
from langchain_community.llms import LlamaCpp
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

In [17]:



system_prefix="""you are an expert in general knowledge. answer the questions asked. 
You have access to the following tools for interacting with the database:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 2 times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
"""

dynamic_few_shot_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\n system answer: {answer}"
    ),
    input_variables=["input"],
    prefix=system_prefix,
    suffix=""
)

from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=dynamic_few_shot_prompt),
        ("human", "{input}"),
        ("system", "{agent_scratchpad}"),
    ]
)


In [18]:
tools = [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())]

prompt_val = full_prompt.invoke(
    {
        "input": "what is capital of France ?",
        "tool_names" : [tool.name for tool in tools],
        "tools" : [tool.name + " - " + tool.description.strip() for tool in tools],
        "agent_scratchpad": [],
    }
)

print(prompt_val.to_string())

System: you are an expert in general knowledge. answer the questions asked. 
You have access to the following tools for interacting with the database:

['wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.']

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of ['wikipedia']
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat 2 times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


User input: what is capital of France ?
 system answer: The capital of France is Paris

User input: what is capital of Canada ?
 system answer: The capital of Canada is Ottawa
Human: what is capital of France ?
S

In [27]:
import os
cwd = os.getcwd()
cwd

'c:\\Users\\Admin\\DISHA\\LEARNING\\projects\\AgenticSystems\\LanchainAgents'

In [34]:

import os
dirname = os.getcwd()
filename = os.path.join(dirname, 'react.gbnf')
filename

'c:\\Users\\Admin\\DISHA\\LEARNING\\projects\\AgenticSystems\\LanchainAgents\\react.gbnf'

In [35]:
os.path.exists(filename)

True

In [38]:


grammar_path = filename#"LanchainAgents\\react.gbnf"

llm = LlamaCpp(grammar_path=grammar_path, model_path=llm.model_path)

tools = [WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1))]

llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from C:\Users\Admin\.cache\huggingface\hub\models--lmstudio-community--Meta-Llama-3.1-8B-Instruct-GGUF\snapshots\8601e6db71269a2b12255ebdf09ab75becf22cc8\.\Meta-Llama-3.1-8B-Instruct-Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str      

root ::= root_5 root_6 
root_1 ::= think act observe 
think ::= [T] [h] [o] [u] [g] [h] [t] [:] [ ] thought [<U+000A>] 
act ::= act_9 
observe ::= [O] [b] [s] [e] [r] [v] [a] [t] [i] [o] [n] [:] [ ] observation [<U+000A>] 
root_5 ::= root_1 root_5 | 
root_6 ::= think final 
final ::= final_13 
thought ::= str 
act_9 ::= [A] [c] [t] [i] [o] [n] [:] [ ] action [<U+000A>] [A] [c] [t] [i] [o] [n] [ ] [I] [n] [p] [u] [t] [:] [ ] action-input [<U+000A>] 
action ::= str 
action-input ::= str 
observation ::= str 
final_13 ::= [F] [i] [n] [a] [l] [ ] [A] [n] [s] [w] [e] [r] [:] [ ] final-answer [<U+000A>] 
final-answer ::= str 
str ::= [0-9A-Za-z., ] str_16 
str_16 ::= [0-9A-Za-z., ] str_16 | 


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | 
Model metadata: {'general.name': 'Meta Llama 3.1 8B Instruct', 'general.architecture': 'llama', 'general.type': 'model', 'llama.block_count': '32', 'general.basename': 'Meta-Llama-3.1', 'general.finetune': 'Instruct', 'general.size_label': '8B', 'general.license': 'llama3.1', 'llama.context_length': '131072', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '14336', 'llama.attention.head_count': '32', 'tokenizer.ggml.eos_token_id': '128009', 'general.file_type': '18', 'llama.attention.head_count_kv': '8', 'llama.rope.freq_base': '500000.000000', 'quantize.imatrix.entries_count': '224', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.vocab_size': '128256', 'llama.rope.dimension_count': '128', 'tok

In [39]:

agent = create_react_agent(llm, tools, full_prompt)

executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

resp = executor.invoke({"input": "what is capital of germany?"})
print(resp)



> Entering new AgentExecutor chain...



llama_print_timings:        load time =    1470.03 ms
llama_print_timings:      sample time =    1910.76 ms /    35 runs   (   54.59 ms per token,    18.32 tokens per second)
llama_print_timings: prompt eval time =   26878.95 ms /   214 tokens (  125.60 ms per token,     7.96 tokens per second)
llama_print_timings:        eval time =   10778.09 ms /    34 runs   (  317.00 ms per token,     3.15 tokens per second)
llama_print_timings:       total time =   39941.54 ms /   248 tokens


Thought: I need to find out which city is the capital of Germany.
Action: the action to take, should be one of wikipedia
Action Input: berlinthe action to take, should be one of wikipedia is not a valid tool, try one of [wikipedia].

Llama.generate: 213 prefix-match hit, remaining 62 prompt tokens to eval

llama_print_timings:        load time =    1470.03 ms
llama_print_timings:      sample time =    2278.95 ms /    38 runs   (   59.97 ms per token,    16.67 tokens per second)
llama_print_timings: prompt eval time =    8559.31 ms /    62 tokens (  138.05 ms per token,     7.24 tokens per second)
llama_print_timings:        eval time =   12352.89 ms /    37 runs   (  333.86 ms per token,     3.00 tokens per second)
llama_print_timings:       total time =   23675.14 ms /    99 tokens


Thought:  I was going to use wikipedia but since it did not work i am going to try wikipidia again 
Action: wikipedia
Action Input: capital of Germany Page: Capital of Germany
Summary: The capital of Germany is the City Land of Berlin. It is the seat of the Federal President of Germany, whose official residence is Bellevue Castle. The Federal Council is the representation of the Lands of Germany and has its seat at the former Prussian House of Lords. Though most of the ministries are seated in Berlin, some of them, as well as some minor departments, are seated in Bonn, the former capital of Germany (western). Although Berlin is officially the capital of Germany, 8,000 out of the 18,000 total officials employed at the federal bureaucracy still work in Bonn, about 600 km (370 mi) away from Berlin.



Llama.generate: 274 prefix-match hit, remaining 184 prompt tokens to eval

llama_print_timings:        load time =    1470.03 ms
llama_print_timings:      sample time =    3423.50 ms /    54 runs   (   63.40 ms per token,    15.77 tokens per second)
llama_print_timings: prompt eval time =   26687.44 ms /   184 tokens (  145.04 ms per token,     6.89 tokens per second)
llama_print_timings:        eval time =   16911.83 ms /    53 runs   (  319.09 ms per token,     3.13 tokens per second)
llama_print_timings:       total time =   47693.63 ms /   237 tokens


Thought: I now know that The Capital Of Germany is the city of Land of Berlin.
Final Answer: The final answer to the question is The capital of Germany is the City Land of Berlin.  It is the seat of the Federal President of Germany, whose official residence

> Finished chain.
{'input': 'what is capital of germany?', 'output': 'The final answer to the question is The capital of Germany is the City Land of Berlin.  It is the seat of the Federal President of Germany, whose official residence'}
